In [1]:
import numpy as np
import os
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle

# Original Settings

In [2]:
elems0,lattices,labor = mli.readInputfile('mli.in.origin')

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [3]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [4]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)

In [5]:
labor = ['iotaline','mapout','fin']

# thick sext 2 thin multipole

In [6]:
elemList,latticeList=mli.sext2thin(elems,lattices,brho=0.50204778184582999)

In [7]:
mli.writeInputfile(elemList,latticeList,labor)
mli.run()
M,G=mli.readTransferMap()

In [8]:
M

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [9]:
G.head()

,exponents,GP
28,f( 30 00 00 ),-1.254486
29,f( 21 00 00 ),-2.937953
33,f( 20 00 01 ),0.971029
34,f( 12 00 00 ),-2.099518
38,f( 11 00 01 ),0.513714


In [10]:
indexThin = []
k2lList = []
for i,item in enumerate(elemList):
    if 'thlm' == item.elem:
        indexThin.append(i)
        k2lList.append(item.k2l)
nThin = len(indexThin)
print(nThin)

18


# Check optimized restult

In [11]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
print(k)

1.4544633270832747


In [12]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
G2_ref

,exponents,GP
28,f( 30 00 00 ),0.000000
29,f( 21 00 00 ),0.000000
33,f( 20 00 01 ),0.727236
34,f( 12 00 00 ),0.000000
38,f( 11 00 01 ),0.000000
39,f( 10 20 00 ),0.000000
40,f( 10 11 00 ),0.000000
43,f( 10 02 00 ),0.000000
48,f( 10 00 02 ),0.000000
49,f( 03 00 00 ),0.000000


In [13]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2

In [14]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0

In [15]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0

In [16]:
print(len(G2_ref),len(G3_ref),len(G4_ref),len(G5_ref))

18 37 65 109


In [17]:
#%%
def objFunc(arg): 
   
    for i,j in enumerate(indexThin):
        elemList[j]['k2l']=arg[i]
        
    mli.writeInputfile(elemList,latticeList,labor)
    mli.run()
    M,G = mli.readTransferMap()
    
    obj = 1.0e30* np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
         +1.0e22* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2) \
         +1.0e12* np.sum((G.loc[210:450,'GP'].values-G4_ref['GP'].values)**2) \
         +        np.sum((G.loc[462:910,'GP'].values-G5_ref['GP'].values)**2)
    return obj

In [18]:
objFunc(k2lList+[0]*nThin)
M0,G0 = mli.readTransferMap()

In [19]:
import pickle
with open('result.thin.sext','rb') as fp:
    result=pickle.load(fp)

In [20]:
result

     fun: 2.1643912463845405e+31
     jac: array([ 1.28145424e+28,  2.61208778e+26, -1.00655452e+28, -1.29433453e+28,
        1.45196052e+28,  1.19908340e+28,  2.85077856e+26,  8.32895715e+27,
        9.87189038e+27,  1.13738409e+28,  3.96361803e+27, -2.26004140e+28,
        2.00275075e+27, -9.12834608e+27, -1.03452187e+28, -1.13927560e+28,
       -1.47718068e+26,  2.44455388e+27])
 message: 'Optimization terminated successfully.'
     nit: 16
 success: True
       x: array([-63.63060194,  -6.58721497,  17.92803175, -50.07063493,
        16.30566301,  -1.9507182 , -49.35581077,   7.1745276 ,
        17.59666097,  17.71108718, -22.31556646, -24.37769313,
       -28.34299493,  25.01246222,  74.30971019, -47.54007517,
       -95.87955716, -35.25459958])

In [21]:
print(objFunc(result.x))
M,G = mli.readTransferMap()

2.1643912465841257e+31


In [22]:
G0.head(10)

,exponents,GP
28,f( 30 00 00 ),-1.254486
29,f( 21 00 00 ),-2.937953
33,f( 20 00 01 ),0.971029
34,f( 12 00 00 ),-2.099518
38,f( 11 00 01 ),0.513714
39,f( 10 20 00 ),-0.788949
40,f( 10 11 00 ),-0.711479
43,f( 10 02 00 ),-0.182533
48,f( 10 00 02 ),-0.628755
49,f( 03 00 00 ),-0.392796


In [23]:
G.head(10)

,exponents,GP
28,f( 30 00 00 ),1.600341
29,f( 21 00 00 ),-1.557497
33,f( 20 00 01 ),-0.397776
34,f( 12 00 00 ),0.813293
38,f( 11 00 01 ),0.657720
39,f( 10 20 00 ),0.050497
40,f( 10 11 00 ),-0.361018
43,f( 10 02 00 ),0.558724
48,f( 10 00 02 ),0.312456
49,f( 03 00 00 ),-0.092801
